In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import datetime

import googletrans

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [24]:
def url_request():
    url= 'https://www.rt.com'
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    cookies= driver.find_element(By.PARTIAL_LINK_TEXT,'Accept cookies').click()
    return driver

In [25]:
driver = url_request()

## Function to get DF

In [26]:
def key_word(word):
    
    search_bar = driver.find_element(By.CLASS_NAME, "js-search-input")
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)

    lst_link=[]
    year_control=[]

    while len(year_control)< 1 and len(lst_link)< 20:
        lnks       = driver.find_elements(By.TAG_NAME,"a")
        lst_rt     = [lnk.get_attribute('href') for lnk in lnks]
        lst_rt_1   = [url for url in lst_rt if 'https://www.rt.com/news/5' in str(url)]
        lst_link   = [i for i in lst_rt_1]
        
        more_button= driver.find_element(By.ID,'listingBtn')
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)",more_button)
        more_button.click()
                
        lst_link=list(set(lst_link))
                
        list_soup = []
        for url in lst_link:
            html  = requests.get(url)
            soup  = BeautifulSoup(html.content, "html.parser")
            list_soup.append(soup)
        art_list = []  
        for soup in list_soup:
            article = soup.getText()
            article = article[article.find('Home'):]
            article = article[:article.find('You can share this story on social media:')]
            article = article.replace('\xa0','').replace('\n','').replace('READ MORE','').replace('/','').replace('HomeWorld News','')
            art_list.append(article)

        day_list=[]
        month_list=[]
        year_list=[]
        year_control=[]

        for soup in list_soup:
            date_str = soup.find('span', {'class': 'date'}).text.strip()
            day, month, year, time = date_str.split()
            day = day.zfill(2)
            month = month.zfill(2).replace(',','')
            year= year
            
            day_list.append(day)
            month_list.append(month)
            year_list.append(year)
            year_control=[i for i in year_list if i == '2020']
                    
    dict_={
        'link': lst_link,
        'article': art_list,
        'year' : year_list,
        'month': month_list,
        'day': day_list
    }
    df= pd.DataFrame(dict_)
    return df

In [27]:
df_usa=key_word('usa')

In [28]:
df_usa.head()

,link,article,year,month,day
0,https://www.rt.com/news/569244-trump-hints-thi...,Trump hints at third-party run in 2024...,2022,Dec,31
1,https://www.rt.com/news/570557-iraq-and-big-am...,"‘If this is not evil, then evil has no...",2023,Jan,28
2,https://www.rt.com/news/570363-poland-fm-ukrai...,Poland considered partitioning Ukraine...,2023,Jan,23
3,https://www.rt.com/news/569877-germany-repair-...,Nord Stream must be repaired – German ...,2023,Jan,14
4,https://www.rt.com/news/571238-us-nord-stream-...,Glenn Diesen: There's only one way to ...,2023,Feb,09


In [29]:
df_united_states=key_word('united states')

In [30]:
df_united_states.head()

,link,article,year,month,day
0,https://www.rt.com/news/572558-north-korea-ove...,North Korea warns of ‘overwhelming’ re...,2023,Mar,07
1,https://www.rt.com/news/572471-trump-china-cov...,Trump demands Covid ‘reparations’ from...,2023,Mar,05
2,https://www.rt.com/news/572537-us-desantis-cal...,"DeSantis visits California, targets ‘w...",2023,Mar,06
3,https://www.rt.com/news/572580-havana-syndrome...,"‘Havana syndrome’ has been debunked, b...",2023,Mar,07
4,https://www.rt.com/news/572464-washington-stup...,American leaders are ‘fundamentally st...,2023,Mar,04


In [33]:

df_us= pd.concat([df_united_states,df_usa])

In [34]:
len(df_us)

47

In [36]:
df_us.drop_duplicates(inplace=True)

In [37]:
len(df_us)

47